In [17]:
import langchain
from langchain_community.document_loaders import DataFrameLoader
import json
import pandas as pd

In [22]:
class_data = pd.read_json('data.json').T

coi_df = class_data['coi_data'].apply(pd.Series)
gpa_df = class_data['gpa_distro'].apply(pd.Series)

# Merge back into original dataframe (if needed)
class_data = class_data.drop(columns=['coi_data']).join(coi_df)
class_data = class_data.drop(columns=['gpa_distro']).join(gpa_df)
class_data.columns = class_data.columns.astype(str)
gpa_df = class_data[[str(i) for i in range(31, 41)]].applymap(lambda x: x['count'] if isinstance(x, dict) else None)
gpa_df.columns = [f"gpa_count_{col}" for col in gpa_df.columns]
class_data = class_data.drop(columns=[str(i) for i in range(31, 41)]).join(gpa_df)
class_data["course_description"] = class_data["course_description"].fillna("(No description)")

keep_columns = ['course_description', 'course_title', 'department_abbrev', 'course_id']
data = class_data[keep_columns]

C:\Users\lucas\AppData\Local\Temp\ipykernel_9908\3533572691.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gpa_df = class_data[[str(i) for i in range(31, 41)]].applymap(lambda x: x['count'] if isinstance(x, dict) else None)


In [ ]:
loader = DataFrameLoader(
    data,
    page_content_column = 'course_description'
)


documents = loader.load()


print(documents[0])




page_content='(No description)' metadata={'course_title': 'Introduction to Anthropology', 'department_abbrev': 'ANTH', 'course_id': 'ANTH 100'}


In [ ]:
#Setting up kwargs

query_prompt = 'Given a search query, retrieve relevant passages that relate to the query'

query_kwargs = {
    'prompt': query_prompt,
    
}

In [ ]:
embedder = HuggingFaceEmbeddings(model_name = 'Alibaba-NLP/gte-Qwen2-7B-instruct',
                                query_encode_kwargs = query_kwargs)




embedded_docs = embedder.embed_documents(documents)

